<table>
  <tr>
    <th>
      <img src="https://raw.githubusercontent.com/riinbre-bioinfo/Colab_Biomarkers/main/Biomarkers/images/RIINBRE-Logo.jpg", height = "125", alt="RI-INBRE Logo">
    </th>
    <th>
      <img src="https://raw.githubusercontent.com/riinbre-bioinfo/Colab_Biomarkers/main/Biomarkers/images/MIC_Logo.png", height = "125", alt="RI-INBRE Logo">
    </th>
  </tr>
</table>

# Analysis of Biomedical Data for Biomarker Discovery
## Submodule 6: Linear and Logistic Regression for Comparison of Quantitative Biomarkers
### Dr. Christopher L. Hemme
### Director, [RI-INBRE Molecular Informatics Core](https://web.uri.edu/riinbre/mic/)
### The University of Rhode Island College of Pharmacy
Last Updated: June 5, 2023

---

## Introduction

In Chapter 2 we introduced the mouse renal ischaemia-reperfusion injury (IRI) model.  In this chapter, we will compare two common biomarkers, serum creatinine (SCr) and blood urea nitrogen (BUN) using linear and logistic regression.  Each of these biomarkers are well-established indicators of kidney injury with relatively well-defined cutoffs between healthy and injured states.  By examining how two well-established biomarkers correlate with each other, you should come away from this chapter with an understanding of how an unknown biomarker could be evaluated compared to an established biomarker.  For this chapter, we will treat SCr as our "standard" biomarker and will use it as the reference to assess the biomarker properties of BUN.

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> Blue boxes will indicate helpful tips.</div>

<div class="alert alert-block alert-warning">
<b>&#127891; Note:</b> Used for interesting asides or notes.
</div>

<div class="alert alert-block alert-success">
<b>&#9997; Reference:</b> This box indicates a reference for an attached figure or table.
</div>

<div class="alert alert-block alert-danger">
<b>&#128721; Caution:</b> A red box indicates potential hazards or pitfalls you may encounter.
</div>

---

## Evaluating Biomarkers Using Linear Regression

A common situation in biomedical research is determining the correlation or relationship between two quantitative variables. Linear regression is a statistical method that allows you to predict a quantitative response. 


First, the installation of packages.  Reminder that this may take a few minutes.

In [ ]:
packages <- c("glm2", "fastDummies", "pROC", "ggpubr")
installed_packages <- packages %in% rownames(installed.packages())
if (any(installed_packages == FALSE)) {install.packages(packages[!installed_packages])}

In [ ]:
require('ggpubr')
require('tidyverse')
require('dplyr')
require('magrittr')
require('glm2')
require('pROC')

In [ ]:
# factoextra visualizes results of multivariate data analysis
# This must be done after the require() statements in order to avoid overwriting certain packages
if(!require(devtools)) install.packages("devtools")
devtools::install_github("kassambara/factoextra")
library("factoextra")

Now we load the experimental object from the last chapter which you downloaded to your local system.  First, create a folder on the left hand side of the Google Colab screen and name it "data".  If you save it somewhere else, you will have to change the code below to reflect the new directory.

In [ ]:
exp_obj <- readRDS(file = "data/exp_obj.rds")
exp_obj

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b>
    <br>SCr = Serum Creatinine (mg/dL)
    <br>BUN = Blood Urea Nitrogen (mg/dL)
</div>

A benefit of using an experimental object is that we can use it to store the raw, unmodified data and then copy that data to another variable for actual manipulation.  Maintaining your raw data in a read-only format (with multiple backups) is good data science practice in general because it ensures that your original data will not be destroyed by the manipulations, which in turn promotes reproducibility.  For the sake of simplicity, we're going to join the biomarker and metadata into a single data frame called <i>biomarker_df</i> using <i>dplyr</i>'s <i>left_join</i> function, which will join the metadata to the biomarker data based on the <b>Sample</b> column.

In [ ]:
biomarker_df <- inner_join(exp_obj$data$biomarkers, exp_obj$metadata, by = 'Sample')
biomarker_df

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> There are many ways to combine data frames and matrices in R.  Common methods are <b>rbind</b> (row bind) or <b>cbind</b> (column bind) which simply concatenate two data structures by row or column, respectively, if the dimensions match (i.e. same number of rows or columns, respectively).  We used a join because we were combining based on a common column <i>Sample</i>.  We specifically used a left join which says "Find all rows in biomarkers that have corresponding sample names in metadata and combine them, adding <b>NULL</b> for any missing data".  There is also right join (match metadata to biomarkers adding <b>NULL</b> for missing data), inner join (match only sample names in both, ignoring missing data), or full join (match all from both adding <b>NULL</b> for missing data).
</div>

Now we'll create a scatter plot of the data using <i>ggplot</i> which we will store in the <i>SCr_scatter</i> variable.  Note two aspects of this chain of functions.  First is the <i>%>%</i> pipe function which pipes <i>biomarker_df</i> into <i>ggplot</i>.  Using this pipe command, <i>ggplot</i> automatically inserts <i>biomarker_df</i> in as the first argument to <i>ggplot</i>.  The second aspect is the layered nature of <i>ggplot</i>, with each layer connected by the <i>+</i> sign.  The first layer creates a blank plot and sets the aesthetic (<i>aes</i> argument) which defines the x and y axes.  The second layer creates a scatterplot using the <i>geom_point</i> geometry, which has its own aesthetic defining the point color and shape.  Finally, we modify the themes to change the colors and font sizes.  This is the <i>tidyverse</i> way of plotting data.  We could certainly run each separate function and save the results to a variable, building the plot in steps.  There are times where this can be useful, such as when you want to create variations of a basic plot.  The advantage of chaining the commands is that you can create the plot in logical chunks depending on your particular needs.

In [ ]:
# create a scatter plot, set the aesthetic (aes), set the colors and shapes of the data points by the Treatment factor levels, label the axes, and make the plot pretty
SCr_scatter <- biomarker_df %>%
    ggplot(aes(x = SCr, y = BUN)) +
        geom_point(size = 3, aes(color = Treatment, shape = Treatment)) +
        xlab("SCr (mg/dL)") +
        ylab("BUN (mg/dL)") +
        theme_bw() +
        theme(text=element_text(size = 24))
SCr_scatter

We can see from the scatter plot that there appears to be a strong positive correlation between SCr and BUN.  However, we have some extreme values at the high concentrations of the biomarkers.  These data points represent extreme kidney injury that may not be treatable.  Because of this, they may not be useful to our analysis.  Let's remove these outliers using the <b>dplyr</b> <i>filter</i> function and replot the data.

In [ ]:
# remove values with SCR >= 2 mg/dL or BUN >= 200 mg/dL and replot the data
biomarker_noOutliers_df <- biomarker_df %>%
    filter(SCr <= 2) %>%
    filter(BUN <= 200)
SCr_scatter_noOutliers <- biomarker_noOutliers_df %>%
    ggplot(aes(x = SCr, y = BUN)) +
        geom_point(size = 3, aes(color = Treatment, shape = Treatment)) +
        xlab("SCr (mg/dL)") +
        ylab("BUN (mg/dL)") +
        theme_bw() +
        theme(text=element_text(size = 24))
SCr_scatter_noOutliers

Because we saved our plot in a variable, we can add new layers to it instead of writing out the entire <i>ggplot</i> data from before.  We will use <i>geom_smooth</i> to add a regression line and 95% confidence intervals.

In [ ]:
SCr_scatter_noOutliers +
    geom_smooth(method="lm")

While this is interesting, what we really care about is the differences between the treatment groups, particularly between the trep treated group and the placebo group.  Let's visualize the data as box plots with the data separated by treatment groups.

In [ ]:
ggplot(biomarker_noOutliers_df, aes(x = Treatment, y = SCr, fill = Treatment)) +
    geom_boxplot(aes(group = Treatment)) +
    ylab("SCr (mg/dL)") +
    theme_bw() +
    theme(text=element_text(size = 24))
ggplot(biomarker_noOutliers_df, aes(x = Treatment, y = BUN, fill = Treatment)) +
    geom_boxplot(aes(group = Treatment)) +
    ylab("BUN (mg/dL)") +
    theme_bw() +
    theme(text=element_text(size = 24))

For both biomarkers, the control and sham groups are fairly similar while the concentrations of both are elevated in in the placebo group.  The trep treated samples showed lower mean concentration of both biomarkers, though not to the level of the control/sham groups. The trep group trending toward healthy or control is expected as this treatment has shown to improve disease state in the literature. The effect also seems to be more prominent with SCr than with BUN.  From the scatter plots and box plots, we now have an idea of what our regression results will look like.  Let's start with a simple linear regression ignoring the Treatment effect to validate our observation from the scatter plot, i.e. that the two biomarkers are positively correlated.  Let's run it first with the data WITH outliers.

In [ ]:
# Run a simple linear regression using BUN as the dependent variable and SCr as the independent variable and produce diagnostics plots

biomarkers_lm <- lm(BUN ~ SCr, data = biomarker_df)
plot(biomarkers_lm)

These four plots are common diagnostic plots for linear regression.  They can be interpreted as follows:

- The residuals vs. fitted plot gives us a measure of the <b>homoscedasticity</b> of the data, that is, is there equal variance in the different groups.  If the data is randomly distributed around 0, then the variances are probably equal.  If there is a fanning effect from left to right, the variances are probably not equal.  Other patterns can indicate non-linear relationships between the data.  Our data appears to be heteroscedastic and at least one of the outliers (point 37) is clear.

- The Q-Q plot indicates if the data is normally distributed.  If the data points fall on the 45 degree line, the data is normally distributed.  Different deviations from this pattern can indicate skewed, tailed or bimodal distributions.  Our data doesn't look bad but again the outliers are obvious (hello again 37).

- The scale location plot is another test of homoscedasticity and like the residuals vs fitted plot, we're looking for a horizontal line with randomly distributed data.  Again, our data looks like the variances are not equal and 37 is still taunting us.

- The residuals vs. leverage plot tells us which points are likely to significantly affect the coefficients of our regression model if they were removed.  The dotted lines represent the <b>Cook Distances</b>.  Anything outside of these lines is a potential troublemaker.  I imagine you can already guess who we're talking about.

Let's run the regression again without the outliers.

In [ ]:
#remove outliers and plot the linear model
biomarkers_noOutliers_lm <- lm(BUN ~ SCr, data = biomarker_noOutliers_df)
plot(biomarkers_noOutliers_lm)

Our residuals vs. fitted, Q-Q, and residuals vs. leverage plots look much better now without 37 mucking things up.  Our scale location plot is still a bit messy, but it's good enough for our purposes.  Let's move on and look at the results of our regression.

<div class="alert alert-block alert-warning">
<b>&#127891; Note:</b> There are ways to address the heteroscedasticity, such as log or square root transformation.  For the purposes of this module, we will forge ahead as is, but feel free to play with the data on your own to see how different transformation affect that plots. 
</div>

The <i>summary</i> function is very useful for analyzing linear regression, ANOVA and similar data.

In [ ]:
summary(biomarkers_noOutliers_lm)

Under estimate we see slope as ~60 and the intercept as ~15. The results show that for every 1 unit increase in SCr, BUN increases by ~60 units.  However, our adjusted R-squared value is low indicating that there is some variability in our data.  Overall, we see that there is a strong correlation between SCr and BUN concentrations, with small changes in SCr correlating to larger changes in BUN.  This suggests that BUN can likely be used as a biomarker (as we already know of course).  A more important question for our purposes, however, is whether it can distinguish between the placebo (diseased) state and the treated state.  To test that, we will need to include the Treatment state in our regression model.  Let's start by plotting out data again, this time breaking out regression line up by group.

<div class="alert alert-block alert-info">
<b>&#9995; Tip:</b> Run the model with the outliers and see how it affects the results.</div>

In [ ]:
biomarker_noOutliers_df %>%
    ggplot(aes(x=SCr, y=BUN, group=Treatment, color=Treatment)) +
    geom_point() +
    geom_smooth(method="lm", aes(fill=Treatment)) +
    theme_bw() +
    theme(text=element_text(size = 24))

<div class="alert alert-block alert-warning">
    <b>&#127891; Alternative Plotting Strategy: <i>ggpubr</i></b><br>We'll recreate the above graph using the <i>ggpubr</i> package which is particularly useful for creating publication quality figures.  You're encouraged to explore the <i>ggpubr</i> documentation to figure out what is happening in each layer.
</div>

In [ ]:
SCr_scatter_ggpubr <-
    ggscatter(
        biomarker_noOutliers_df,
        x = "SCr", y = "BUN",
        color = "Treatment",
        shape = "Treatment", size = 4,
        add = "reg.line",
        add.params = list(color="Treatment", fill = "Treatment"),
        conf.int = TRUE,
        ggtheme = theme_pubr()
    ) +
    grids(linetype = "dashed") +
    stat_regline_equation(
        label.x.npc = "left", 
        label.y.npc = "middle",
        size = 6
    ) +
    stat_cor(
        method="pearson", 
        size=6,
        aes(
            color=Treatment,
            label = paste(after_stat(rr.label), ..p.label.., sep = "~`,`~")
        )) +
    font("xlab",size=24) +
    font("ylab",size=24) +
    font("xy.text",size=18) +
    font("legend.text", size=20) +
    theme(legend.title= element_blank())

ggpar(
    SCr_scatter_ggpubr,
    xlab = "SCr (mg/dL)",
    ylab = "BUN (mg/dL)"
)

Now we see that the slope changes significantly based on group.  The control and sham groups show almost no change in BUN levels between samples while the placebo and trep treated groups show larger positive relationships.  Let's now go to the regression model using BUN as the dependent variable.

In [ ]:
biomarkers_noOutliers_lm <- lm(BUN ~ Treatment, data = biomarker_noOutliers_df)
summary(biomarkers_noOutliers_lm)

The intercept term represents the mean of the control group, which is ~33 mg/dL BUN.  We add the coefficients for each factor to this value to get each groups' mean, so SHAM = 33.102 - 3.628 = 29.474, PLB = 33.102 + 49.520 = 82.622, TRE = 33.102 + 33.549 = 66.651.  Both placebo and treated group means are elevated compared to the control, and both have significant <i>p</i>-values.  BUN can likely distinguish between healthy and injured states, and may be able to distinguish the treated and placebo states.  Let's compare the results to SCr.

In [ ]:
biomarkers_noOutliers_lm <- lm(SCr ~ Treatment, data = biomarker_noOutliers_df)
summary(biomarkers_noOutliers_lm)

For SCr, both placebo and and treated states have significant p-values, but based on our scatter plot, it looks like the two states separate better by SCr values than BUN.  It thus seems likely that SCr is more sensitive to state changes, a result that is consistent with clinical consensus.

---

## Evaluating Biomarkers Using Logistic Regression

A common situation in biomedical research is comparing results between two specific states, for example, control vs. disease or placebo vs. treatment.  In these situations, we can employ a binary classification scheme for our two states in which the state is represented by a binary dummy variable consisting of 0 and 1.  Linear regression will not work for such a system, but logistic regression works very well.  Common classification schemes include:

<ul>
    <li>Classification by experiment design (e.g. placebo vs. control)</li>
    <li>Classification by clinical diagnosis (e.g. healthy vs. diseased)</li>
    <li>Classification by biomarker standard (e.g. biomarker concentration above cutoff vs. biomarker concentration below cutoff)</li>
</ul>

How you classify your data is up to you, but it should reflect proper experimental design and should address a specific hypothesis.

Let's start our analysis by creating a dummy binary variable.  By convention, we call this variable <i>D</i> with the "healthy" state as 0 and the "diseased" state as 1.  Our first scenario will use an SCr concentration cutoff to establish our binary variable (SCr >= 1 is our injured state).

In [ ]:
#adding D column with values
biomarker_noOutliers_df$D <- ifelse(biomarker_noOutliers_df$SCr >=1 ,1,0)
biomarker_noOutliers_df

This classification ignores our experimental groups but let's refer back to our scatter plot to predict what the results might look like.

<div class="alert alert-block alert-info">
    <b>&#9995; Tip:</b> <b>If...Else</b> blocks are a fundamental tool of logic control in programming.  The <i>ifelse</i> function is a quick and dirty implementation for simple logic circuits.  Basically, this command says "If SCr value is greater than or equal to 1, return the value 1, otherwise return 0".  These values are then stored in the new column <i>D</i>.</div>

In [ ]:
SCr_scatter_noOutliers

An SCR cutoff at 1 mg/dL should distinguish well between the placebo (injured) states and the control/sham states, though some placebo groups that are under that cutoff will likely be falsely assigned.  It also looks like most of the trep treated samples will fall below this cutoff.  BUN, however, tells a different story.  While the control/sham and placebo groups seem to clearly separate around 50 mg/dL, there is much more overlap between the placebo and trep treated groups.  Let's plot the logistic curve using the <i>glm</i> method.

In [ ]:
biomarker_noOutliers_df %>%
  ggplot(aes(BUN, D)) +
  geom_point(alpha = 0.2) +
  geom_smooth(method = "glm", method.args = list(family = "binomial")) +
  labs(
    title = "Logistic Regression Model", 
    x = "BUN mg/dL",
    y = "Probability of kidney injury based on [SCR] >= 1"
    )

The inflection point occurs around BUN 75 mg/dL and we can see that there is overlap between our two states.  Let's run the regression.  BUN will be our independent variable and the dependent variable will always be our binary dummy variable.

<div class="alert alert-block alert-info">
    <b>&#9995; Tip:</b> <b>family</b> defines what is called the <i>link function</i> of our regression model.  In linear regression, the link function is <i>identity</i> (<b>family = "gaussian"</b>).  For logistic regression, the link function is <i>logit</i> (<b>family = "binomial"</b>).</div>

In [ ]:
#plotting the logistic regression by calling the glm function
biomarkers_logistic_SCr1 <- glm(D ~ BUN, data = biomarker_noOutliers_df, family = "binomial")
summary(biomarkers_logistic_SCr1)

A 1 mg/dL increase in BUN is associated with a log odds increase of ~0.06 or an odds increase of ~1.06.  Not a huge change but it is statistically significant.  Now let's see what the ROC curve looks like.

<div class="alert alert-block alert-info">
    <b>&#9995; Tip:</b> <b>ROC</b> stands for <b>receiver operating characteristic</b> and plots the true positive rate against the false positive rate.  It was originally used in World War II to identify and classify radar signals, but has utility in many different fields such as biomarker discovery and assessment of machine learning algorithms.  The <b>area under the curve (AUC)</b> provides an assessment of the scheme, with AUC=1 being perfect classification and AUC=0.5 representing random assortment.</div>

In [ ]:
roc(D ~ BUN, biomarker_noOutliers_df)

In [ ]:
#plotting the ROC curve
roc(biomarker_noOutliers_df$D,
            biomarker_noOutliers_df$BUN, percent=FALSE,
            ci=TRUE, boot.n=100, ci.alpha=0.9, stratified=FALSE,
            plot=TRUE, auc.polygon=TRUE, max.auc.polygon=TRUE, grid=TRUE,
            print.auc=TRUE, show.thres=TRUE)

AUC = 0.871 isn't bad.  This suggests that BUN can be used to classify healthy/treated states from the injured state, but this might just be due to the wide concentration ranges in the placebo group.  

<div class="alert alert-block alert-warning">
<b>&#127891; Self-Assessment:</b> Try setting different SCr cutoffs to see how it affects the results.  Also, try the opposite analysis by checking SCr vs. a BUN concentration cutoff.  How does SCr compare to BUN?  
</div>

Sometimes, we may want to compare molecular data to clinical diagnostic data.  For example, in a clinical trial, patients will be classified based on clinical criteria into categories such as healthy vs. diseased or degree of disease state.  Do these clinical criteria accurately reflect molecular changes in disease state?  It's possible that clinical classifications may represent a "trailing effect".  In other words, the molecular data could indicate a change of state before that state change is visibly obvious. 

We can set up a similar classification scheme with the IRI data.  This time, instead of using a SCr cutoff, we are going to define our groups based on our experimental design by explicitly defining the placebo (PLB) group as the "diseased" group and all other groups as "healthy".  Our hypothesis is that the molecular data from the trep treated samples will more closely resemble those of the healthy samples.  If this is the case, we should see a clear break between the placebo and the treated/healthy samples.  If the break is not clear, it may still give us an indication of how effective the treatment is in bringing biomarker levels in the injured state back to the healthy state.

In [ ]:
biomarker_noOutliers_df$D <- ifelse(biomarker_noOutliers_df$Treatment == "PLB",1,0)
biomarker_noOutliers_df

Now, lets run the logistic regression model. 

In [ ]:
biomarkers_logistic_PLB <- glm(D ~ BUN, data = biomarker_noOutliers_df, family = "binomial")
summary(biomarkers_logistic_PLB)

In [ ]:
biomarker_noOutliers_df %>%
  ggplot(aes(BUN, D)) +
  geom_point(alpha = 0.2) +
  geom_smooth(method = "glm", method.args = list(family = "binomial")) +
  labs(
    title = "Logistic Regression Model", 
    x = "BUN mg/dL",
    y = "Probability of kidney injury based on Trep treatment"
    )

In [ ]:
roc(biomarker_noOutliers_df$D,
            biomarker_noOutliers_df$BUN, percent=FALSE,
            ci=TRUE, boot.n=100, ci.alpha=0.9, stratified=FALSE,
            plot=TRUE, auc.polygon=TRUE, max.auc.polygon=TRUE, grid=TRUE,
            print.auc=TRUE, show.thres=TRUE)

The results are very similar, but they tell us different stories based upon the question (i.e. the classification scheme) we asked.  In the first scenario, we showed that SCr cutoffs can be used to distinguish between the healthy and injured groups and that the treated group tends to approximate the healthy samples, more so for SCr than for BUN.  In the second scenario, we explicitly asked if the treated samples approximated the healthy samples, but we didn't make this determination with respect to SCr.

<div class="alert alert-block alert-warning">
<b>&#127891; Self-Assessment:</b> As before, try the above scenario using SCr as the biomarker, or by classifying TRE vs PLB without the healthy samples. 
</div>

---

### Overview

These results show (as expected) that both SCr and BUN are useful biomarkers that correlate strongly and can distinguish between healthy and injured states.  SCr appears to be more sensitive than BUN, particularly when distinguishing between the placebo and trep treated states.  While these examples are somewhat contrived since we already know the utility of these two biomarkers, these methods can be used to assess any biomarker, such as those identified in omics data.  In the next chapter, we will begin examining proteomics data from the IRI model to identify new potential biomarkers. 

---

<p><span style="font-size: 30px"><b>Quizzes</b></span> <span style="float : inline;">(run the command below to display the quizzes)</span> </p>

In [ ]:
IRdisplay::display_html('<iframe src="quizes/Chapter6_Quizes.html" width=100% height=450></iframe>')

## References

[Shiva N, Sharma N, Kulkarni YA, Mulay SR, Gaikwad AB. Renal ischemia/reperfusion injury: An insight on in vitro and in vivo models. Life Sci. 2020 Sep 1;256:117860. doi: 10.1016/j.lfs.2020.117860. Epub 2020 Jun 11. PMID: 32534037.][shiva]

[Hou J, Tolbert E, Birkenbach M, Ghonem NS. Treprostinil alleviates hepatic mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Nov;143:112172. doi: 10.1016/j.biopha.2021.112172. Epub 2021 Sep 21. PMID: 34560548; PMCID: PMC8550798.][hou]

[Ding M, Tolbert E, Birkenbach M, Gohh R, Akhlaghi F, Ghonem NS. Treprostinil reduces mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Sep;141:111912. doi: 10.1016/j.biopha.2021.111912. Epub 2021 Jul 15. PMID: 34328097; PMCID: PMC8429269.][ding]


[ding]: https://pubmed.ncbi.nlm.nih.gov/34328097/ "Ding M, Tolbert E, Birkenbach M, Gohh R, Akhlaghi F, Ghonem NS. Treprostinil reduces mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Sep;141:111912. doi: 10.1016/j.biopha.2021.111912. Epub 2021 Jul 15. PMID: 34328097; PMCID: PMC8429269."
[hou]: https://pubmed.ncbi.nlm.nih.gov/34560548/ "Hou J, Tolbert E, Birkenbach M, Ghonem NS. Treprostinil alleviates hepatic mitochondrial injury during rat renal ischemia-reperfusion injury. Biomed Pharmacother. 2021 Nov;143:112172. doi: 10.1016/j.biopha.2021.112172. Epub 2021 Sep 21. PMID: 34560548; PMCID: PMC8550798."
[shiva]: https://pubmed.ncbi.nlm.nih.gov/32534037/ "Shiva N, Sharma N, Kulkarni YA, Mulay SR, Gaikwad AB. Renal ischemia/reperfusion injury: An insight on in vitro and in vivo models. Life Sci. 2020 Sep 1;256:117860. doi: 10.1016/j.lfs.2020.117860. Epub 2020 Jun 11. PMID: 32534037."

---